In [167]:
import numpy as np
import pandas as pd

In [168]:
df = pd.read_csv(r"D:\Python\Kaggle_Datasets\CarPrice_prediction\data\cleanned_data.csv")

In [169]:
df['kms_driven']

0       45000
1          40
2       28000
3       36000
4       41000
        ...  
718    132000
719     27000
720     40000
721    Petrol
722    Petrol
Name: kms_driven, Length: 723, dtype: object

In [170]:
df.head(1)

,index,name,company,year,Price,kms_driven,fuel_type
0,0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol


In [171]:
df.head(1)

,index,name,company,year,Price,kms_driven,fuel_type
0,0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol


In [172]:
df = df[df['fuel_type']!="LPG"]

In [173]:
df = df[df['kms_driven']!="Petrol"]

In [174]:
df['kms_driven'] = df['kms_driven'].astype(int)

In [175]:
df['fuel_type'].value_counts()

fuel_type
Petrol    389
Diesel    330
Name: count, dtype: int64

In [176]:
X=df[['name','company','year','kms_driven','fuel_type']]
y=df['Price']

In [177]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [178]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [179]:
ohe=OneHotEncoder()
ohe.fit(X[['name','company','fuel_type']])

OneHotEncoder()

In [180]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']),
                                    remainder='passthrough')

In [181]:
lr=LinearRegression()

In [182]:
pipe=make_pipeline(column_trans,lr)

In [183]:
pipe.fit(X_train,y_train)

d:\Python\Kaggle_Datasets\CarPrice_prediction\venv\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'Petrol'], dtype=object)]),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [184]:
y_pred=pipe.predict(X_test)

In [185]:
r2_score(y_test,y_pred)

0.5363912378279717

In [186]:
scores=[]
for i in range(1000):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [187]:
np.argmax(scores)

np.int64(672)

In [188]:
scores[np.argmax(scores)]

0.8354552538125822

In [189]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=np.argmax(scores))
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)

0.8354552538125822

In [190]:
import pickle
pickle.dump(pipe,open(r"D:\Python\Kaggle_Datasets\CarPrice_prediction\models\model.pkl",'wb'))